# Introduction

### Background
Toronto is Canada's largest city, the fourth largest in North America, and home to a diverse population of about 2.8 million people. It is a global centre for business, finance, arts and culture and is consistently ranked one of the world's most livable cities.
### Problem
When you are looking to open a restaurant, location is the most important part of successful business. This project will segment the neighborhoods of Toronto into major clusters and examine their food. The results will help you to discover culture and variety of the neighborhood.

# Data 

### Toronto City Dataset
Data will be scraped from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. Data is consist of Post Code, Borough,  and Neighborhood.

### Geographical Coordinates
Here is a link to a csv file that has the geographical coordinates of each postal code of Toronto City link: http://cocl.us/Geospatial_data

### Foursquare API
Foursquare API, a location data provider, will be used to make RESTful API calls to retrieve data about venues in different neighborhoods.